In [1]:
from mip import *
import numpy as np
import json
import pandas as pd

In [2]:
constructors = json.load(open('./data/constructors.json'))
drivers = json.load(open('./data/drivers.json'))

In [3]:
BUDGET = 100
DRIVERS, TEAMS = 21, 10
DRIVERS_AMOUNT, TEAMS_AMOUNT = 5, 2

In [5]:
names = [d['driver'] for d in drivers]
team = [c['team'] for c in constructors]
new_budget = BUDGET

SEASON_POINTS = 0

for race in range(len(drivers[0]['races'])):
    driver_race_points = []
    team_race_points = []
    
    #  first race based on initial budget
    if race == 0:
        driver_price = [d['startPrice'] for d in drivers]
        team_price = [c['startPrice'] for c in constructors]
    # next races based on the maximal potential budget
    else:
        driver_price = [d['races'][race-1]['price'] for d in drivers]
        team_price = [c['races'][race-1]['price'] for c in constructors]
        
        # BUDGET = new_budget

    driver_race_points.append([d['races'][race]['points']['total'] for d in drivers])
    team_race_points.append([c['races'][race]['points']['total'] for c in constructors])

    # dataframes
    drivers_df = pd.DataFrame([names, driver_price, *driver_race_points], ['name', 'price', 'points']).T
    constructors_df = pd.DataFrame([team, team_price, *team_race_points], ['name', 'price', 'points']).T

    df = pd.concat([drivers_df, constructors_df], ignore_index=True)
    
    # define input data to linear programming
    ROW = df.shape[0]
    COL = df.shape[1]-1

    all_indexes = np.arange(0, ROW*COL, 1).reshape(ROW, COL)
    driver_indexes = np.arange(0, (ROW-TEAMS)*COL, 1).reshape(DRIVERS, COL)
    team_indexes = np.arange(DRIVERS*COL, ROW*COL, 1).reshape(TEAMS, COL)

    matrix = df.to_numpy()
    matrix = matrix[:, 1:].astype(float)
    data = np.array(matrix).flatten()

    # linear programming calculation
    m = Model(sense=MAXIMIZE, solver_name=CBC)

    y = [m.add_var(var_type=BINARY) for i in range(ROW)]
    # MINIMUM PRICE 
    m.objective = minimize(xsum(y[i] * data[all_indexes[i, 0]] for i in range(ROW)))
    # MAXIMUM POINTS 
    m.objective = maximize(xsum(y[i] * data[all_indexes[i, 1]] for i in range(ROW)))

    # NUMBER OF DRIVERS
    m += xsum(y[i] for i in range(DRIVERS)) == DRIVERS_AMOUNT
    # NUMBER OF TEAMS
    m += xsum(y[i] for i in range(DRIVERS, ROW)) == TEAMS_AMOUNT
    # BUDGET
    m += xsum(data[all_indexes[i, 0]]*y[i] for i in range(ROW)) <= new_budget

    m.optimize()
    squad = [i for i in range(ROW) if y[i].x >= 0.99]

    squad_data = df.iloc[squad]

    if race == 0:
        price_diff = driver_price = [d['races'][race]['price']-d['startPrice'] for d in drivers if d['driver'] in list(squad_data['name'])]
        price_diff = round(np.sum(price_diff), 2)
    else:
        price_diff = driver_price = [d['races'][race]['price']-d['races'][race-1]['price'] for d in drivers if d['driver'] in list(squad_data['name'])]
        price_diff = round(np.sum(price_diff), 2)

    # show info about price, points, best driver
    drivers_squad = squad_data.iloc[0:5]
    turbo_driver = drivers_squad[drivers_squad['points']== drivers_squad['points'].max()]

    print(f'Race: {drivers[0]["races"][race]["circuit"]}')
    print(f'Budget: {new_budget} mln')
    print('Squad')
    print(squad_data)
    print('')
    print(f'Turbo driver: {turbo_driver["name"].values[0]}')
    print(f'Price: {squad_data["price"].sum()}')
    print(f'Points (no boost): {squad_data["points"].sum()}')
    print(f'Points with turbo driver: {squad_data["points"].sum() + turbo_driver["points"].values[0]}')
    print('-----------------------------------------')
    
    SEASON_POINTS += squad_data["points"].sum() + turbo_driver["points"].values[0]

    if race == 0:
        price_diff = driver_price = [d['races'][race]['price']-d['startPrice'] for d in drivers if d['driver'] in list(squad_data['name'])]
        price_diff = round(np.sum(price_diff), 2)
    else:
        price_diff = driver_price = [d['races'][race]['price']-d['races'][race-1]['price'] for d in drivers if d['driver'] in list(squad_data['name'])]
        price_diff = round(np.sum(price_diff), 2)
        
    new_budget = new_budget + price_diff

print(f'Season points: {SEASON_POINTS}')

Race: Bahrain
Budget: 100 mln
Squad
               name price points
0    Max Verstappen  26.9     35
1   Fernando Alonso   8.3     39
3       Zhou Guanyu   4.9     15
5      Sergio Perez    18     28
10     Lance Stroll   7.5     17
21         Red Bull  27.2     78
22     Aston Martin   6.7     52

Turbo driver: Fernando Alonso
Price: 99.5
Points (no boost): 264
Points with turbo driver: 303
-----------------------------------------
Race: Saudi Arabia
Budget: 100.5 mln
Squad
               name price points
0    Max Verstappen    27     61
1   Fernando Alonso   8.5     23
3       Zhou Guanyu   4.9      6
15     Yuki Tsunoda   4.8      9
16    Nyck De Vries     5      7
21         Red Bull  27.3     95
23          Ferrari  22.1     59

Turbo driver: Max Verstappen
Price: 99.6
Points (no boost): 260
Points with turbo driver: 321
-----------------------------------------
Race: Australia
Budget: 100.8 mln
Squad
               name price points
0    Max Verstappen  27.1     36
1   Fernando